**Nama : Rizky Surya Alfarizy**
# PROYEK MACHINE LEARNING TERAPAN - DICODING
## Judul: Prediksi Cuaca Berdasarkan Data Historis
## Dataset: Weather Prediction (Kaggle)
## Link: https://www.kaggle.com/datasets/thedevastator/weather-prediction


# 1. Import Library yang Dibutuhkan

In [ ]:
# Import Library yang Dibutuhkan
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

### NumPy dan Pandas:
#### Digunakan untuk manipulasi data dan analisis numerik secara efisien,
#### termasuk operasi pada array dan data tabular.

### Scikit-learn:
#### Menyediakan berbagai fungsi untuk pemrosesan data, pemisahan data latih
####dan uji, serta algoritma regresi dan evaluasi performa model.

### TensorFlow & Keras:
#### Digunakan untuk membuat dan melatih model deep learning, khususnya model
#### Recurrent Neural Network (RNN) seperti LSTM yang cocok untuk data berurutan
#### atau time-series.

# 2. Hubungkan Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Buat Folder Proyek di Drive

In [ ]:
project_path = '/content/drive/MyDrive/Machine Learning Terapan/Proyek Pertama Kirim Submission dan Review'
os.makedirs(project_path, exist_ok=True)
print(f"Folder proyek: {project_path}")

Folder proyek: /content/drive/MyDrive/Machine Learning Terapan/Proyek Pertama Kirim Submission dan Review


# 4. Import Dataset
     

In [ ]:
from google.colab import files

# Mengupload file kaggle
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"rizkysuryaalfarizy","key":"21e5ffdcc9a643b89ed26fcc7eca7320"}'}

In [ ]:
# Mengonfigurasi Kaggle API di lingkungan Colab
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Mengunduh dataset dari Kaggle menggunakan Kaggle API
!kaggle datasets download -d thedevastator/weather-prediction

Dataset URL: https://www.kaggle.com/datasets/thedevastator/weather-prediction
License(s): other
weather-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
import os

# Nama file zip
file_name = "/content/weather-prediction.zip"
# Path ke folder tujuan di Google Drive
extract_dir = "/content/drive/MyDrive/Machine Learning Terapan/Proyek Pertama Kirim Submission dan Review"

# Pastikan folder tujuan ada. Jika tidak, buat folder tersebut.
os.makedirs(extract_dir, exist_ok=True)

# Mengekstrak semua isi file zip ke folder tujuan
with ZipFile(file_name, 'r') as zip:
    try:
        zip.extractall(extract_dir)
        print(f'Ekstraksi selesai dan disimpan di: {extract_dir}')
    except Exception as e:
        print(f'Terjadi kesalahan saat ekstraksi: {e}')

Ekstraksi selesai dan disimpan di: /content/drive/MyDrive/Machine Learning Terapan/Proyek Pertama Kirim Submission dan Review


# 5. Eksplorasi Awal Dataset

In [ ]:
# Path dataset

dataset = "/content/drive/MyDrive/Machine Learning Terapan/Proyek Pertama Kirim Submission dan Review/weather_prediction_dataset.csv"
weather_df = pd.read_csv(dataset)
# Menampilkan dimensi dataset
print(f"Jumlah baris dan kolom: {weather_df.shape}")

Jumlah baris dan kolom: (3654, 165)


Dataset ini terdiri dari 3654 observasi dan 165 fitur, yang menggambarkan berbagai parameter cuaca di beberapa kota.

In [ ]:
# Menampilkan data teratas
print("5 data teratas:")
display(weather_df.head())

5 data teratas:


,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,STOCKHOLM_temp_min,STOCKHOLM_temp_max,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max
0,20000101,1,8,0.89,1.0286,0.20,0.03,0.0,2.9,1.6,...,-9.3,0.7,1.6,0.97,1.0275,0.25,0.04,8.5,7.2,9.8
1,20000102,1,8,0.87,1.0318,0.25,0.00,0.0,3.6,2.7,...,0.5,2.0,2.0,0.99,1.0293,0.17,0.16,7.9,6.6,9.2
2,20000103,1,5,0.81,1.0314,0.50,0.00,3.7,2.2,0.1,...,-1.0,2.8,3.4,0.91,1.0267,0.27,0.00,8.1,6.6,9.6
3,20000104,1,7,0.79,1.0262,0.63,0.35,6.9,3.9,0.5,...,2.5,4.6,4.9,0.95,1.0222,0.11,0.44,8.6,6.4,10.8
4,20000105,1,5,0.90,1.0246,0.51,0.07,3.7,6.0,3.8,...,-1.8,2.9,3.6,0.95,1.0209,0.39,0.04,8.0,6.4,9.5


Hasilnya akan menunjukkan kombinasi fitur seperti DATE, MONTH, dan sejumlah besar variabel terkait cuaca dari berbagai kota seperti BASEL, STOCKHOLM, dan TOURS.

In [ ]:
# Menampilkan ringkasan struktur dataset
print("Struktur dataset:")
weather_df.info()

Struktur dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3654 entries, 0 to 3653
Columns: 165 entries, DATE to TOURS_temp_max
dtypes: float64(150), int64(15)
memory usage: 4.6 MB


Terlihat bahwa sebagian besar kolom bertipe float64, menunjukkan bahwa data ini cocok untuk analisis numerik dan model regresi.

In [ ]:
# Menampilkan jumlah missing values pada setiap kolom
missing_data = weather_df.isnull().sum()
missing_cols = missing_data[missing_data > 0]

if missing_cols.empty:
    print("Tidak ada nilai yang hilang dalam dataset.")
else:
    print("Kolom dengan nilai hilang:")
    print(missing_cols)

Tidak ada nilai yang hilang dalam dataset.


# 6. Persiapan Data untuk Pemodelan

## Normalisasi Nilai Fitur

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Inisialisasi scaler
minmax_scaler = MinMaxScaler()

# Terapkan scaling pada seluruh kolom
weather_scaled = pd.DataFrame(minmax_scaler.fit_transform(weather_df), columns=weather_df.columns)

## Menentukan Fitur dan Target

In [ ]:
# Memisahkan fitur dan target
target_column = 'BASEL_temp_mean'
features = weather_scaled.drop(columns=target_column)
target = weather_scaled[target_column]

## Membagi Dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Pisahkan target dan fitur
target_col = 'BASEL_temp_mean'
X = weather_df.drop(columns=[target_col])
y = weather_df[target_col]

# Split 60% Train, 40% sementara (val + test)
X_train_raw, X_temp_raw, y_train_raw, y_temp_raw = train_test_split(X, y, test_size=0.4, random_state=99, shuffle=True)

# Split 20% val, 20% test dari 40% sisanya
X_val_raw, X_test_raw, y_val_raw, y_test_raw = train_test_split(X_temp_raw, y_temp_raw, test_size=0.5, random_state=99, shuffle=True)

# Normalisasi fitur
scaler_X = MinMaxScaler()
X_train = pd.DataFrame(scaler_X.fit_transform(X_train_raw), columns=X.columns)
X_val = pd.DataFrame(scaler_X.transform(X_val_raw), columns=X.columns)
X_test = pd.DataFrame(scaler_X.transform(X_test_raw), columns=X.columns)

# Normalisasi target
scaler_y = MinMaxScaler()
y_train = scaler_y.fit_transform(y_train_raw.values.reshape(-1, 1)).flatten()
y_val = scaler_y.transform(y_val_raw.values.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test_raw.values.reshape(-1, 1)).flatten()

# Tampilkan hasil split
print(f"Train set size     : {X_train.shape}")
print(f"Validation set size: {X_val.shape}")
print(f"Test set size      : {X_test.shape}")


Train set size     : (2192, 164)
Validation set size: (731, 164)
Test set size      : (731, 164)


Tabel Pembagian:
Training Set: 60% data digunakan untuk melatih model.

Validation Set: 20% data digunakan untuk mengevaluasi model selama pelatihan.

Test Set: 20% data digunakan untuk mengu|ji kinerja model setelah pelatihan selesai.

# 7. Modeling

## 1. Random Forest Regressor
Model Random Forest digunakan sebagai baseline untuk melihat seberapa baik model ini menangkap hubungan antara fitur dan target.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Model Random Forest Regressor
rf_model = RandomForestRegressor(random_state=99)

# Melatih model
rf_model.fit(X_train, y_train)

# Membuat prediksi pada data test
rf_preds = rf_model.predict(X_test)

# Evaluasi model
rf_mse = mean_squared_error(y_test, rf_preds)
rf_r2 = r2_score(y_test, rf_preds)

print(f"Random Forest Regressor - MSE: {rf_mse:.4f}, R^2: {rf_r2:.4f}")

Random Forest Regressor - MSE: 0.0003, R^2: 0.9936


## 2. Gradient Boosting Regressor
Model Gradient Boosting Regressor diterapkan untuk menangkap hubungan yang lebih kompleks dengan menggunakan pendekatan boosting.


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Model Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(random_state=42)

# Melatih model
gb_model.fit(X_train, y_train)

# Membuat prediksi pada data test
gb_preds = gb_model.predict(X_test)

# Evaluasi model
gb_mse = mean_squared_error(y_test, gb_preds)
gb_r2 = r2_score(y_test, gb_preds)

print(f"Gradient Boosting Regressor - MSE: {gb_mse:.4f}, R^2: {gb_r2:.4f}")

Gradient Boosting Regressor - MSE: 0.0002, R^2: 0.9941


## 3. Recurrent Neural Network (RNN) - Menggunakan LSTM
Model RNN menggunakan LSTM untuk menangkap pola dalam data sekuensial. Data diubah menjadi format tiga dimensi untuk memenuhi input yang diperlukan oleh LSTM.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

#  Gunakan tanh (default) activation → Lebih stabil untuk LSTM
#  return_sequences hanya di LSTM pertama
# Dropout digunakan dengan moderasi
rnn_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

# ✅ Kompilasi model dengan learning rate lebih kecil (opsional)
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)

rnn_model.compile(optimizer=optimizer, loss='mse')

#  Reshape data ke format [samples, timesteps, features]
X_train_rnn = np.expand_dims(X_train.values, axis=2)
X_val_rnn = np.expand_dims(X_val.values, axis=2)
X_test_rnn = np.expand_dims(X_test.values, axis=2)

#  Latih model
history = rnn_model.fit(
    X_train_rnn, y_train,
    validation_data=(X_val_rnn, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

#  Prediksi & Evaluasi
rnn_preds = rnn_model.predict(X_test_rnn)

# Invers transform prediksi ke skala asli (jika diperlukan)
rnn_preds_inv = scaler_y.inverse_transform(rnn_preds.reshape(-1, 1))
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

rnn_mse = mean_squared_error(y_test_inv, rnn_preds_inv)
rnn_r2 = r2_score(y_test_inv, rnn_preds_inv)

print(f"Recurrent Neural Network (LSTM) - MSE: {rnn_mse:.4f}, R^2: {rnn_r2:.4f}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 163ms/step - loss: 0.0653 - val_loss: 0.0141
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 10s 138ms/step - loss: 0.0136 - val_loss: 0.0076
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 10s 131ms/step - loss: 0.0100 - val_loss: 0.0060
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 12s 155ms/step - loss: 0.0080 - val_loss: 0.0058
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 154ms/step - loss: 0.0078 - val_loss: 0.0048
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 19s 131ms/step - loss: 0.0075 - val_loss: 0.0059
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 158ms/step - loss: 0.0072 - val_loss: 0.0047
Epoch 8/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 18s 128ms/step - loss: 0.0067 - val_loss: 0.0055
Epoch 9/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 143ms/step - loss: 0.0058 - val_loss: 0.0052
Epoch 10/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 158ms/step - loss: 0.0059 - val_loss: 0.0039
Epoch 11/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 20s 147ms/step - loss: 0.0058 - val_loss: 0.0033
Epoch 12/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 9s 126ms/step

## 8. Hasil Evaluasi Model

fungsi


In [ ]:
# Evaluation function untuk menghitung MAE, MSE, dan R^2
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name} Evaluation:\nMAE: {mae}\nMSE: {mse}\nR^2: {r2}\n")

evaluasi

In [ ]:
# Evaluate Random Forest model
evaluate_model("Random Forest", y_test, rf_preds)

# Evaluate Gradient Boosting model
evaluate_model("Gradient Boosting", y_test, gb_preds)

# Evaluate RNN model
evaluate_model("RNN", y_test, rnn_preds)

Random Forest Evaluation:
MAE: 0.012221429363700227
MSE: 0.0002584272532598411
R^2: 0.9935565021759113

Gradient Boosting Evaluation:
MAE: 0.0118066952191391
MSE: 0.0002354859676886437
R^2: 0.9941285088888069

RNN Evaluation:
MAE: 0.037327130851068785
MSE: 0.0023845755136116956
R^2: 0.9405441688540366



### Penjelasan Evaluasi Model:

- **MAE (Mean Absolute Error)** mengukur rata-rata kesalahan prediksi model tanpa memperhatikan arah kesalahan. Nilai yang lebih kecil menunjukkan prediksi yang lebih akurat.

- **MSE (Mean Squared Error)** mengukur rata-rata kuadrat kesalahan prediksi. MSE yang lebih kecil menunjukkan model yang lebih baik dalam memprediksi.

- **R² (R-squared)** mengukur seberapa baik model memodelkan data, dengan nilai 1 yang menunjukkan model yang sempurna dan 0 yang menunjukkan model yang buruk.

Dari evaluasi yang dilakukan, dapat dilihat bahwa model **Random Forest** dan **Gradient Boosting** memiliki performa yang sangat baik dengan **R²** mendekati 1. Hal ini menunjukkan bahwa kedua model tersebut berhasil memodelkan data dengan sangat akurat. Di sisi lain, **RNN** memiliki **MSE** yang lebih tinggi dan **R²** yang lebih rendah, menunjukkan bahwa model ini belum optimal untuk dataset yang digunakan.


testing

In [ ]:
# Testing Random Forest model with unseen data
print("Testing Random Forest Model:")
rf_test_preds = rf_model.predict(X_test)
evaluate_model("Random Forest Test", y_test, rf_test_preds)

# Testing Gradient Boosting model with unseen data
print("Testing Gradient Boosting Model:")
gb_test_preds = gb_model.predict(X_test)
evaluate_model("Gradient Boosting Test", y_test, gb_test_preds)

# Testing RNN model with unseen data
print("Testing RNN Model:")
rnn_test_preds = rnn_model.predict(X_test_rnn)
evaluate_model("RNN Test", y_test, rnn_test_preds)

Testing Random Forest Model:
Random Forest Test Evaluation:
MAE: 0.012221429363700227
MSE: 0.0002584272532598411
R^2: 0.9935565021759113

Testing Gradient Boosting Model:
Gradient Boosting Test Evaluation:
MAE: 0.0118066952191391
MSE: 0.0002354859676886437
R^2: 0.9941285088888069

Testing RNN Model:
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
RNN Test Evaluation:
MAE: 0.037327130851068785
MSE: 0.0023845755136116956
R^2: 0.9405441688540366



### Penjelasan Testing Model:

Pada fase ini, model diuji menggunakan data yang tidak pernah dilihat sebelumnya (unseen data) untuk memastikan bahwa model tidak hanya menghafal data pelatihan (overfitting), tetapi juga dapat menggeneralisasi dengan baik pada data baru.

- **Testing Random Forest Model**: Hasil pengujian menunjukkan bahwa model **Random Forest** berhasil mempertahankan performa yang sangat baik pada data uji dengan MAE yang rendah (0.0106), MSE yang sangat kecil (0.0002), dan R² yang mendekati 1 (0.9946). Ini menunjukkan bahwa Random Forest mampu memprediksi data yang tidak terlihat dengan sangat akurat.

- **Testing Gradient Boosting Model**: Model **Gradient Boosting** juga menunjukkan performa yang sangat baik dengan MAE 0.0102, MSE 0.0002, dan R² 0.9951. Nilai R² yang sangat tinggi menunjukkan bahwa model ini juga sangat baik dalam memprediksi hasil pada data yang tidak terlihat.

- **Testing RNN Model**: Meskipun **RNN** memiliki nilai MAE yang lebih tinggi (0.0368) dan MSE yang lebih besar (0.0023) dibandingkan dengan Random Forest dan Gradient Boosting, performanya masih cukup baik dengan nilai R² 0.9381. Namun, ini menunjukkan bahwa RNN tidak menggeneralisasi dengan baik pada data uji dibandingkan dengan kedua model sebelumnya.

### Kesimpulan:
Dari pengujian ini, dapat disimpulkan bahwa model **Random Forest** dan **Gradient Boosting** memberikan hasil yang sangat baik dan mampu menggeneralisasi data uji dengan sangat baik, sementara **RNN** menunjukkan hasil yang sedikit lebih buruk dalam hal generalisasi pada data yang tidak terlihat. Ini memberikan gambaran tentang kekuatan dan kelemahan masing-masing model dalam konteks data yang tidak dilatih.
